# Intro

Jon Messier

3/1/2023

# Applying Advanced Transformations (Core)

## The Data 

You will be working with a heavily modified version of the Superheroes dataset from Kaggle.

The dataset includes two csv's:

- `superhero_info.csv` : Contains Name, Publisher, Demographic Info, and Body measurements.

- `superhero_powers.csv`: Contains Hero name and list of powers
---
## The Task

Your task is two-fold:

### I. Clean the files and combine them into one final DataFrame.

- This dataframe should have the following columns:
    - Hero (Just the name of the Hero)
    - Publisher
    - Gender
    - Eye color
    - Race
    - Hair color
    - Height (numeric)
    - Skin color
    - Alignment
    - Weight (numeric)
    - Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
      - Agility
      - Flight
      - Superspeed
      - etc.

Hint: There is a space in "100 kg" or "52.5 cm"


### II. Use your combined DataFrame to answer the following questions.

-  Compare the average weight of super powers who have Super Speed to those who do not.
- What is the average height of heroes for each publisher?

In [1]:
import pandas as pd
import numpy as np
import os, json

## Import Data

In [2]:
info = pd.read_csv('Data/superhero_info - superhero_info.csv')
powers = pd.read_csv('Data/superhero_powers - superhero_powers.csv')
display(info.head())
display(powers.head())

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


## Cleanup Hero Info data
- [x] Split Hero/Publisher
- [x] Break up Height/Weight list, drop units
- [x] Convert Height/Weight to float


In [3]:
#Split Hero/Publisher into separate columns
info[['Hero','Publisher']] = info['Hero|Publisher'].str.split('|', expand=True)
#Drop the original Hero|Publisher column
info.drop(columns='Hero|Publisher', inplace=True)
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [4]:
#Look at Measurements column
mes = info.loc[0,'Measurements']
mes

"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [5]:
#Replace the single quotes with doubles
# to put in JSON format
info['Measurements'] = info['Measurements'].str.replace("'",'"')
#Update column using json Load string (json.loads)
info['Measurements'] =info['Measurements'].apply(json.loads)
info['Measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [6]:
#Convert the measurements to a series 
mes = info['Measurements'].apply(pd.Series)
#Add the series to the original dataframe
info = pd.concat((info, mes), axis=1)
#drop orginal column
info.drop(columns="Measurements", inplace=True)
info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


In [7]:
#Split the units in the Height and Weight columns to make them numeric
info[['Height(cm)',"Hunits"]] = info['Height'].str.split(" ",expand=True)
info[['Weight(kg)',"Wunits"]] = info['Weight'].str.split(" ",expand=True)

#Drop split Height and Weight units
info.drop(columns=["Hunits","Wunits"], inplace=True)

#Convert to numeric
info['Height(cm)']= info['Height(cm)'].astype('float')
info['Weight(kg)']= info['Weight(kg)'].astype('float')

#Drop original columns
info.drop(columns=['Height','Weight'], inplace=True)

#Review changes
display(info.head())
info.info()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height(cm),Weight(kg)
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Gender      463 non-null    object 
 1   Race        463 non-null    object 
 2   Alignment   463 non-null    object 
 3   Hair color  463 non-null    object 
 4   Eye color   463 non-null    object 
 5   Skin color  463 non-null    object 
 6   Hero        463 non-null    object 
 7   Publisher   463 non-null    object 
 8   Height(cm)  463 non-null    float64
 9   Weight(kg)  463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


## Cleanup hero superpower data
- [x] Split up powers csv
- [x] Explode into row entries
- [x] Use a loop to create colums and O.H.E. powers with T/F-values in new columns

In [8]:
powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [9]:
#Split the comma-separated Powers column into lists
powers["Powers_split"] = powers['Powers'].str.split(',')
print(type(powers.loc[1,"Powers_split"]))
powers["Powers_split"].head()

<class 'list'>


0      [Agility, Super Strength, Stamina, Super Speed]
1    [Accelerated Healing, Durability, Longevity, S...
2    [Agility, Accelerated Healing, Cold Resistance...
3                                 [Lantern Power Ring]
4    [Accelerated Healing, Intelligence, Super Stre...
Name: Powers_split, dtype: object

In [10]:
#Explode the powers list.  Basically create a row for every power
exp = powers.explode("Powers_split")
exp[["hero_names", "Powers", "Powers_split"]].head()

,hero_names,Powers,Powers_split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Agility
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Strength
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Stamina
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",Super Speed
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",Accelerated Healing


In [11]:
## saving the unique values from the exploded column
cols_to_make = exp['Powers_split'].dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [12]:
#Create a new column for every unique power
#Compare the column name to the original string of powers. 
for col in cols_to_make:
    powers[col] = powers['Powers'].str.contains(col)
powers.head()

C:\Users\jonme\AppData\Local\Temp\ipykernel_9032\396298350.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col] = powers['Powers'].str.contains(col)
C:\Users\jonme\AppData\Local\Temp\ipykernel_9032\396298350.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers[col] = powers['Powers'].str.contains(col)
C:\Users\jonme\AppData\Local\Temp\ipykernel_9032\396298350.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

,hero_names,Powers,Powers_split,Agility,Super Strength,Stamina,Super Speed,Accelerated Healing,Durability,Longevity,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",False,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance...",True,True,True,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring],False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre...",False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [13]:
#Drop the original and list of powers columns
powers.drop(columns=["Powers",'Powers_split'], inplace=True)

## Merge data

In [14]:
#Compare datasets
print(f'Hero Info dataframe shape: {info.shape}')
print(f'Hero Power dataframe shape: {powers.shape}')

Hero Info dataframe shape: (463, 10)
Hero Power dataframe shape: (667, 168)


In [15]:
#Merge the info and powers dataframes
#Left merge. Only add powers from heros who are in both 
# info(left) and powers(right) dataframes
merged = pd.merge(info,powers,left_on="Hero", right_on="hero_names", how="left")

In [16]:
#Check Nan values.  If this were an OUTER merge there would be NAN 
#Hero names
merged.isna().sum().sum()

0

In [17]:
merged.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height(cm),Weight(kg),...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


In [18]:
#Drop the redundant hero_names
merged.drop(columns='hero_names', inplace=True)

## II. Questions

In [19]:
#Compare the average weight of super heros who have Super Speed to those who do not.
merged.groupby("Super Speed")["Weight(kg)"].mean()

Super Speed
False    101.773585
True     129.404040
Name: Weight(kg), dtype: float64

In [20]:
#What is the average height of heroes for each publisher?
merged.groupby("Publisher")["Height(cm)"].mean()

Publisher
DC Comics            181.923913
Dark Horse Comics    176.909091
George Lucas         159.600000
Image Comics         211.000000
Marvel Comics        191.546128
Shueisha             171.500000
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Name: Height(cm), dtype: float64